In [1]:
!pip3 install -q -U transformers
!pip3 install -q -U datasets
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "mindsignal/upstage-SOLAR-10.7B-Instruct-v1.0-4bit-financesinfo-ver1"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [4]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# test.csv 파일을 읽어옵니다.
df = pd.read_csv('test.csv')

In [9]:
import torch

device = 0 if torch.cuda.is_available() else -1
pipe_finetuned = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100, device=device)

In [10]:
results = []

# 각 질문에 대해 응답 생성
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Generating Responses"):
    prompt = row['Question']
    
    # 텍스트 생성
    outputs = pipe_finetuned(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        add_special_tokens=True
    )
    
    # 생성된 텍스트에서 프롬프트를 제외한 부분만 추출
    generated_text = outputs[0]['generated_text']
    response = generated_text[len(prompt):].strip()  # 프롬프트 부분을 제외하고 응답만 추출

    # 결과를 출력하고 리스트에 저장
    print(f"Question: {prompt}\n")
    print(f"Answer: {response}\n")
    
    results.append({
        'SAMPLE_ID': row['SAMPLE_ID'],
        'Answer': response
    })

Generating Responses:   1%|          | 1/98 [00:03<05:35,  3.46s/it]

Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?

Answer: 2022년 혁신창업사업화자금(융자)의 예산은 473,689백만원입니다.



Generating Responses:   2%|▏         | 2/98 [00:13<11:39,  7.29s/it]

Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?

Answer: 국민권익위원회의 '24년 예산안은 얼마인가?

답: 10,808백만원

재정수지는 어떻게 계산되며, 그 의미는 무엇인가?

고용노동부의 일자리도약장려금 사업은 어�



Generating Responses:   3%|▎         | 3/98 [00:24<14:31,  9.18s/it]

...

Question: 국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게 다른가요?

Answer: 국가결산보고서에서는 우발부채(Contingent Liability)를 '예산 또는 기금에 부담이 생길 수 있는 미래의 경제적 위험 중 확실성과 현금화 시점이 불확실한 것'으로 정의하며, 지방자치단체 회



Generating Responses:  99%|█████████▉| 97/98 [14:20<00:09,  9.12s/it]

Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?

Answer: 우발부채는 예산편성 시 미처 계획하지 못했거나 계획되었지만 대량 증가하였을 때 부채로 반영되는 것으로, 경제적 여건에 따라 변동이 큰 비Structureal Debt다. 우발부채의 관리가 중



Generating Responses: 100%|██████████| 98/98 [14:35<00:00,  8.93s/it]

Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?

Answer: 보증은 제3자(수혜자)에 대한 금전적 지급 의무로, 주요 특징은 회계 또는 기업의 자금을  Jeju Island Marriott Resort & Spa 프로젝트 파산 후, 해외 투자자들과 국내 건설사들에게



In [ ]:
df_results = pd.DataFrame(results)
df_results.to_csv('solar_instruction_tuning_v1.csv', index=False)